In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
# with implementation from http://mnemstudio.org/path-finding-q-learning-tutorial.htm

In [3]:
#use trainning data to build the recommendation
data=pd.read_csv('/Users/irwang/Desktop/aa228_final_project/final_labeled_data_train.csv')
data.head()

,Unnamed: 0,user_id,s,a,sp,r,movie_num
0,0,133,12,1,11,-3.0,1
1,1,133,12,2,11,-2.0,1
2,2,133,12,3,11,-3.0,1
3,3,133,12,4,11,-3.0,1
4,4,133,12,5,11,-3.0,1


In [5]:
test=pd.read_csv('/Users/irwang/Desktop/aa228_final_project/final_labeled_data_test.csv')
test.head()
test_u=test[test['r']>0][['user_id','s','sp','r']].drop_duplicates()

In [6]:
#make it consistent with index
data['s']=data['s']-1
data['a']=data['a']-1
data['sp']=data['sp']-1
data.head()
avg_r=data.groupby(['s', 'a']).r.agg(['mean']).reset_index()
avg_r[avg_r['mean']>0]

,s,a,mean
15,1,1,0.545455


In [7]:
def update_q(s,sp,a,alpha,gamma):
    rsa = r[s, a]
    qsa = q[s, a]
    new_q = qsa + alpha * (rsa + gamma * max(q[sp, :]) - qsa)
    q[s, a] = new_q
    return r[s, a]

In [20]:
def learning(alpha,gamma):
    rewards=[]
    epsilon = 1.0 
    for e in range(int(n_episodes)):
        #print e
        states = np.arange(1,14)
        random_state.shuffle(states)
        current_state = states[0]
        total_rewards = 0
        for i in range(steps):
            if np.random.rand(1)[0] >= epsilon:
                action = np.argmax(q[current_state])
            else:
                action = np.random.randint(0,14)
            next_state=int(data[(data['s']==current_state)&(data['a']==action)].sample()['sp'])
            reward = update_q(current_state, next_state, action,alpha=alpha, gamma=gamma) 
            total_rewards += reward
            current_state = next_state
        epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*e) 
        #print epsilon
        rewards.append(total_rewards)
    return rewards

In [25]:
r = avg_r.pivot(index='s', columns='a', values='mean').as_matrix()
n_episodes = 1000
max_epsilon = 1
min_epsilon = 0.05
decay_rate = 0.01  
random_state = np.random.RandomState(1999)
steps=150    

/Users/irwang/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [ ]:
for al in np.arange(0.1,0.35,0.05):
    # defines the reward/connection graph
    q = np.full(r.shape, -100)
    rewards_list=learning (alpha=al,gamma=0.99)
    rec=[]
    for i in range(14):
        d={}
        d['s']=i+1
        d['recommend']=np.argmax(q[i])+1
        rec.append(d.copy())
    recommender=pd.DataFrame(rec)
    pred=pd.merge(test_u,recommender)
    print al,len(pred[pred['sp']==pred['recommend']])/float(len(pred)),len(pred[(pred['sp']==pred['recommend'])&(pred['r']>3)])/float(len(pred[pred['r']>3]))